In [37]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [64]:
# Downloading S&P 500 data for the past 3 years for 5 stocks
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0] # Taking the ticker symbols from wikipedia
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-') # Yahoo Finance uses dashes instead of dots
tickers = sp500['Symbol'].unique().tolist() # Get all unique tickers
end_date = pd.to_datetime("2024-03-31")
start_date = end_date - pd.DateOffset(years=3)
df = yf.download(tickers, start=start_date, end=end_date)
adj_close_df = df['Adj Close']
adj_close_df

[*********************100%%**********************]  503 of 503 completed


2 Failed downloads:
['SOLV', 'GEV']: Exception("%ticker%: Data doesn't exist for startDate = 1617163200, endDate = 1711857600")


,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-31,124.520882,23.900000,119.893715,94.851738,187.940002,113.139778,38.369999,263.910522,475.369995,146.842529,...,219.365051,30.818001,124.099350,60.656418,49.291420,101.519333,102.440552,151.905182,485.179993,153.522751
2021-04-01,125.059547,23.860001,120.728020,95.114670,188.240005,112.290108,39.410000,265.907227,483.339996,151.860992,...,223.879227,31.891434,126.801689,60.911766,50.668716,100.998123,103.974609,151.829285,491.779999,153.922424
2021-04-05,127.515709,24.230000,123.574432,93.028671,186.690002,113.158669,39.619999,268.572601,491.619995,154.805847,...,225.853592,31.804867,129.741592,62.115582,49.882957,102.716179,104.770050,154.391373,492.799988,154.429367
2021-04-06,128.535828,24.059999,123.878700,92.362549,190.029999,113.262512,39.930000,269.050232,491.339996,153.120346,...,226.936600,32.151138,134.987854,62.161198,49.741692,102.803055,108.027534,155.160034,491.160004,151.982407
2021-04-07,126.868317,23.930000,125.537506,92.213524,179.940002,113.394684,40.270000,269.948273,493.410004,150.980392,...,227.942978,31.882786,131.335266,62.060883,49.962414,102.252884,107.999123,154.438812,485.309998,153.912689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-22,147.210846,14.820000,172.046646,176.798874,167.860001,110.024338,90.680000,336.198242,499.519989,193.509995,...,272.332977,35.610001,99.889999,52.270000,113.489998,129.210007,136.229996,126.961258,291.440002,168.419785
2024-03-25,145.323914,14.920000,170.618591,176.878128,167.990005,109.467110,90.989998,332.532440,507.230011,190.630005,...,272.841339,35.490002,99.410004,52.480000,114.650002,127.760002,134.619995,127.669930,297.940002,167.332886
2024-03-26,144.175781,14.920000,169.480133,177.532028,167.389999,110.949753,90.589996,335.092529,507.600006,188.960007,...,273.260010,35.450001,100.580002,51.799999,113.790001,127.900002,134.800003,127.630005,291.380005,165.198959


In [69]:
# Calculating lognormal daily returns
log_returns = np.log(adj_close_df / adj_close_df.shift(1)) # Calculate lognormal daily returns
log_returns = log_returns.dropna(axis=0, how='all') # Drop rows with all NaN values
log_returns

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-01,0.004317,-0.001675,0.006935,0.002768,0.001595,-0.007538,0.026744,0.007537,0.016627,0.033605,...,0.020369,0.034238,0.021542,0.004201,0.027559,-0.005147,0.014864,-0.000500,0.013512,0.002600
2021-04-05,0.019450,0.015388,0.023303,-0.022175,-0.008268,0.007705,0.005314,0.009974,0.016986,0.019206,...,0.008780,-0.002718,0.022920,0.019571,-0.015629,0.016868,0.007621,0.016734,0.002072,0.003288
2021-04-06,0.007968,-0.007041,0.002459,-0.007186,0.017732,0.000917,0.007794,0.001777,-0.000570,-0.010948,...,0.004784,0.010829,0.039640,0.000734,-0.002836,0.000845,0.030618,0.004966,-0.003333,-0.015972
2021-04-07,-0.013058,-0.005418,0.013302,-0.001615,-0.054558,0.001166,0.008479,0.003332,0.004204,-0.014074,...,0.004425,-0.008382,-0.027431,-0.001615,0.004428,-0.005366,-0.000263,-0.004659,-0.011982,0.012621
2021-04-08,0.010384,-0.011770,0.019051,0.008424,0.001333,-0.002751,-0.005228,0.006526,0.012948,0.016607,...,-0.003201,0.005415,0.006086,-0.004861,-0.010481,-0.001984,0.005334,0.010939,0.035603,0.013338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-22,-0.008711,0.002703,0.005296,0.005338,-0.001904,-0.008465,-0.000110,-0.022066,-0.023211,-0.008439,...,-0.004747,0.002249,-0.000701,-0.000574,0.000000,-0.006019,-0.002493,0.001023,-0.012345,0.003440
2024-03-25,-0.012901,0.006725,-0.008335,0.000448,0.000774,-0.005077,0.003413,-0.010964,0.015317,-0.014995,...,0.001865,-0.003376,-0.004817,0.004010,0.010169,-0.011286,-0.011889,0.005566,0.022058,-0.006474
2024-03-26,-0.007932,0.000000,-0.006695,0.003690,-0.003578,0.013453,-0.004406,0.007669,0.000729,-0.008799,...,0.001533,-0.001128,0.011701,-0.013042,-0.007529,0.001095,0.001336,-0.000313,-0.022264,-0.012835


In [70]:
# Creating a covariance matrix
cov_matrix = log_returns.cov()
cov_matrix

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
A,0.000315,0.000157,0.000141,0.000044,0.000209,0.000136,0.000067,0.000152,0.000187,0.000174,...,0.000103,0.000137,0.000153,0.000063,0.000051,0.000162,0.000094,0.000113,0.000220,0.000169
AAL,0.000157,0.000836,0.000213,0.000025,0.000472,0.000093,0.000132,0.000179,0.000220,0.000247,...,0.000143,0.000222,0.000425,0.000030,0.000095,0.000205,0.000124,0.000170,0.000304,0.000132
AAPL,0.000141,0.000213,0.000286,0.000031,0.000246,0.000102,0.000075,0.000175,0.000238,0.000193,...,0.000104,0.000135,0.000178,0.000064,0.000059,0.000143,0.000090,0.000110,0.000227,0.000150
ABBV,0.000044,0.000025,0.000031,0.000159,0.000001,0.000059,0.000043,0.000047,0.000031,0.000038,...,0.000041,0.000030,0.000024,0.000046,0.000043,0.000043,0.000031,0.000050,0.000032,0.000057
ABNB,0.000209,0.000472,0.000246,0.000001,0.000955,0.000115,0.000101,0.000213,0.000318,0.000289,...,0.000135,0.000206,0.000417,0.000011,0.000124,0.000206,0.000146,0.000169,0.000355,0.000188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XYL,0.000162,0.000205,0.000143,0.000043,0.000206,0.000108,0.000098,0.000160,0.000173,0.000170,...,0.000120,0.000142,0.000159,0.000076,0.000064,0.000290,0.000105,0.000117,0.000196,0.000126
YUM,0.000094,0.000124,0.000090,0.000031,0.000146,0.000075,0.000072,0.000093,0.000103,0.000101,...,0.000077,0.000089,0.000124,0.000055,0.000049,0.000105,0.000147,0.000082,0.000132,0.000086
ZBH,0.000113,0.000170,0.000110,0.000050,0.000169,0.000108,0.000082,0.000107,0.000118,0.000119,...,0.000081,0.000105,0.000148,0.000063,0.000061,0.000117,0.000082,0.000256,0.000150,0.000094
ZBRA,0.000220,0.000304,0.000227,0.000032,0.000355,0.000136,0.000094,0.000218,0.000269,0.000281,...,0.000136,0.000229,0.000253,0.000055,0.000103,0.000196,0.000132,0.000150,0.000645,0.000205


In [72]:
# Creating Portfolio Return and Risk functions
def st_dev(weights, cov_matrix):
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

def portfolio_return(weights, log_returns):
    return np.sum(log_returns.mean() * weights) * 252 # 252 trading days in a year, for annual returns

In [73]:
# Creative a negative Sharpe ratio function
def neg_sharpe(weights, log_returns, cov_matrix, risk_free_rate):
    p_return = portfolio_return(weights, log_returns)
    p_risk = st_dev(weights, cov_matrix)
    return -(p_return - risk_free_rate) / p_risk

In [74]:
risk_free_rate = 0.0425 # Long term average of the 10 year US Treasury bond, usually considered risk-free

In [76]:
# Setting constraints and bounds for the optimization function
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}) # Sum of weights should be 1
''' 
Minimum bound = 0 as we are not shorting any stocks
Maximum bound = 0.5 as we are not investing more than 50% in any stock
'''
bounds = [(0, 0.2) for _ in range(len(tickers))]

In [77]:
# Setting initial weights to be equal for all stocks
initial_weights = np.array([1/len(tickers)] * len(tickers))
initial_weights

array([0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198

In [81]:
# Optimizing the weights to maximise the Sharpe ratio
optimized_weights = minimize(neg_sharpe, initial_weights, args=(log_returns, cov_matrix, risk_free_rate), method='SLSQP', bounds=bounds, constraints=constraints)

In [82]:
optimized_weights.x # Optimal weights for the portfolio

array([0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198807,
       0.00198807, 0.00198807, 0.00198807, 0.00198807, 0.00198

In [83]:
# Displaying the optimal portfolio analytics
print("Optimal Portfolio Analytics")
for ticker, weight in zip(tickers, optimized_weights.x):
    print(f"{ticker}: {weight * 100:.2f}%")

optimal_return = portfolio_return(optimized_weights.x, log_returns)
optimal_risk = st_dev(optimized_weights.x, cov_matrix)
sharpe_ratio = (optimal_return - risk_free_rate) / optimal_risk

print(f"Optimal Portfolio Return: {optimal_return * 100:.2f}%")
print(f"Optimal Portfolio Risk: {optimal_risk * 100:.2f}%")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

Optimal Portfolio Analytics
MMM: 0.20%
AOS: 0.20%
ABT: 0.20%
ABBV: 0.20%
ACN: 0.20%
ADBE: 0.20%
AMD: 0.20%
AES: 0.20%
AFL: 0.20%
A: 0.20%
APD: 0.20%
ABNB: 0.20%
AKAM: 0.20%
ALB: 0.20%
ARE: 0.20%
ALGN: 0.20%
ALLE: 0.20%
LNT: 0.20%
ALL: 0.20%
GOOGL: 0.20%
GOOG: 0.20%
MO: 0.20%
AMZN: 0.20%
AMCR: 0.20%
AEE: 0.20%
AAL: 0.20%
AEP: 0.20%
AXP: 0.20%
AIG: 0.20%
AMT: 0.20%
AWK: 0.20%
AMP: 0.20%
AME: 0.20%
AMGN: 0.20%
APH: 0.20%
ADI: 0.20%
ANSS: 0.20%
AON: 0.20%
APA: 0.20%
AAPL: 0.20%
AMAT: 0.20%
APTV: 0.20%
ACGL: 0.20%
ADM: 0.20%
ANET: 0.20%
AJG: 0.20%
AIZ: 0.20%
T: 0.20%
ATO: 0.20%
ADSK: 0.20%
ADP: 0.20%
AZO: 0.20%
AVB: 0.20%
AVY: 0.20%
AXON: 0.20%
BKR: 0.20%
BALL: 0.20%
BAC: 0.20%
BK: 0.20%
BBWI: 0.20%
BAX: 0.20%
BDX: 0.20%
BRK-B: 0.20%
BBY: 0.20%
BIO: 0.20%
TECH: 0.20%
BIIB: 0.20%
BLK: 0.20%
BX: 0.20%
BA: 0.20%
BKNG: 0.20%
BWA: 0.20%
BXP: 0.20%
BSX: 0.20%
BMY: 0.20%
AVGO: 0.20%
BR: 0.20%
BRO: 0.20%
BF-B: 0.20%
BLDR: 0.20%
BG: 0.20%
CDNS: 0.20%
CZR: 0.20%
CPT: 0.20%
CPB: 0.20%
COF: 0.20%
CAH: 